In [24]:
from src.util_tools import compute_cross_entropy, compute_ortho_loss
from tools.project import INPUT_PATH, LOGS_PATH, OUTPUT_PATH

import audiocraft
from audiocraft.models import MusicGen
from audiocraft.utils.notebook import display_audio
import torch
from gradio.cli.commands.components.publish import colors
from omegaconf import DictConfig
from torch import set_grad_enabled
from torch.onnx.symbolic_opset9 import tensor
from torchviz import make_dot
import typing as tp
from audiocraft.modules.conditioners import ConditioningAttributes
import tqdm
import torch
from audiocraft.data.audio import audio_read, audio_write
from audiocraft.data.audio_utils import convert_audio_channels, convert_audio
import umap
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from torch.utils.tensorboard import SummaryWriter
from sklearn.decomposition import PCA
from random import shuffle
from torch.utils.data import TensorDataset, random_split, DataLoader
from audioldm_eval.metrics.fad import FrechetAudioDistance
import os
DEVICE = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
EXAMPLES_LEN = 5
torch.cuda.is_available()

True

In [8]:
fad = FrechetAudioDistance(verbose=True)
model = MusicGen.get_pretrained('facebook/musicgen-small')
model.set_generation_params(
	use_sampling=True,
	top_k=250,
	duration=EXAMPLES_LEN
)

Using cache found in /home/mszawerda/.cache/torch/hub/harritaylor_torchvggish_master
/home/mszawerda/.cache/pypoetry/virtualenvs/musicgen-ufgTm-Qc-py3.10/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Reload fad_generated_folder_cache /home/mszawerda/musical-generative-models-conditioning/data/input/textual-inversion/metal_fad_fad_feature_cache.npy
Loading data to RAM


100%|██████████| 14/14 [00:00<00:00, 237.22it/s]


{'frechet_audio_distance': 30.598606179254183}

In [30]:
res=model.generate([f"music in the style of jazz song"]*5, progress=True)
for a_idx in range(res.shape[0]):
    music = res[a_idx].cpu()
    music = music/np.max(np.abs(music.numpy()))
    path = OUTPUT_PATH("textual-inversion", 'metal', 'temp', f'music_p{a_idx}')
    audio_write(path, music, model.cfg.sample_rate)

In [33]:
fd_score = fad.score(INPUT_PATH('textual-inversion', 'metal_fad'), OUTPUT_PATH("textual-inversion", 'metal', 'temp'))
os.remove(OUTPUT_PATH("textual-inversion", 'metal', 'temp_fad_feature_cache.npy'))
list(fd_score.values())[0]

Reload fad_generated_folder_cache /home/mszawerda/musical-generative-models-conditioning/data/input/textual-inversion/metal_fad_fad_feature_cache.npy
Loading data to RAM


100%|██████████| 5/5 [00:00<00:00, 258.63it/s]


14.55249079201133